In [1]:
import pandas as pd
import numpy as np
import os
import pandas_datareader as pdr
from datetime import datetime


In [2]:
# set time frame
timeframe = '1D'

# start and end date
start_date = datetime(2008,6,1)
end_date = datetime(2009,6,1)

# set tickers
tickers = ['DIS']

# pulling data from pdr 
tickers_df = pdr.DataReader(tickers,'yahoo',start_date,end_date)

In [3]:
# closing prices dataframe
data = pd.DataFrame()

# get closing prices for all tickers in tickers
for ticker in tickers:
    data[ticker]=tickers_df['Adj Close'][ticker]

In [4]:
# add percent change
data['daily_return'] = data['DIS'].pct_change().dropna()

In [5]:
data = data.dropna()

In [6]:
# EWMA Short and Long Windows
short_window = 10
long_window = 50

# Fast and Slow EWMAs from short and long windows
data['ewma_short'] = data['DIS'].ewm(halflife=short_window).mean()
data['ewma_long'] = data['DIS'].ewm(halflife=long_window).mean()

# Construct a crossover trading signal
data['crossover_long'] = np.where(data['ewma_short'] > data['ewma_long'], 1.0, 0.0) 
# making sep columns if (where) fast passes close == long 
data['crossover_short'] = np.where(data['ewma_short'] < data['ewma_long'], -1.0, 0.0)
data['crossover_signal'] = data['crossover_long'] + data['crossover_short']


In [7]:
# Volatility - Short and Long Windows
short_vol_window = 10
long_vol_window = 50

# Construct a `Fast` and `Slow` Exponential Moving Average from short and long windows, respectively
data['short_vol'] = data['daily_return'].ewm(halflife=short_vol_window).std()
data['long_vol'] = data['daily_return'].ewm(halflife=long_vol_window).std()

# Construct a crossover trading signal
data['vol_trend_long'] = np.where(data['short_vol'] < data['long_vol'], 1.0, 0.0)
data['vol_trend_short'] = np.where(data['short_vol'] > data['long_vol'], -1.0, 0.0) 
data['vol_trend_signal'] = data['vol_trend_long'] + data['vol_trend_short']


In [8]:
data = data.dropna()

In [9]:
# Set bollinger band window
bollinger_window = 20

# Calculate rolling mean and standard deviation
data['bollinger_mid_band'] = data['DIS'].rolling(window=bollinger_window).mean()
data['bollinger_std'] = data['DIS'].rolling(window=20).std() # will use for upper and lower bands of bollinger

# Calculate upper and lowers bands of bollinger band
data['bollinger_upper_band']  = data['bollinger_mid_band'] + (data['bollinger_std'] * 1)
data['bollinger_lower_band']  = data['bollinger_mid_band'] - (data['bollinger_std'] * 1)

# Calculate bollinger band trading signal # signal to calc (doing mean revering strategy)
data['bollinger_long'] = np.where(data['DIS'] < data['bollinger_lower_band'], 1.0, 0.0)
data['bollinger_short'] = np.where(data['DIS'] > data['bollinger_upper_band'], -1.0, 0.0)
data['bollinger_signal'] = data['bollinger_long'] + data['bollinger_short']

data.head()

,DIS,daily_return,ewma_short,ewma_long,crossover_long,crossover_short,crossover_signal,short_vol,long_vol,vol_trend_long,vol_trend_short,vol_trend_signal,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band,bollinger_long,bollinger_short,bollinger_signal
Date,,,,,,,,,,,,,,,,,,,
2008-06-04,29.141947,0.035262,28.662836,28.649082,1.0,0.0,1.0,0.027482,0.027482,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2008-06-05,29.260721,0.004076,28.876093,28.855795,1.0,0.0,1.0,0.020440,0.020554,1.0,0.0,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2008-06-06,28.005114,-0.042911,28.635214,28.638682,0.0,-1.0,-1.0,0.032815,0.032287,0.0,-1.0,-1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2008-06-09,28.149338,0.005150,28.524124,28.538081,0.0,-1.0,-1.0,0.028181,0.028053,0.0,-1.0,-1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2008-06-10,28.700790,0.019590,28.558895,28.566148,0.0,-1.0,-1.0,0.026366,0.026360,0.0,-1.0,-1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [10]:
data.to_csv('../Data_2/Disney_Closing_2008.csv',index=True)